# **Modelagem molecular por homologia**

Se observarmos a quantidade de sequências depositadas no **Genbank** (https://www.ncbi.nlm.nih.gov/genbank/statistics/) e compararmos com o número de sequências de proteínas depositadas no **Protein Data Bank** (https://www.rcsb.org), nos deparamos com uma enorme discrepância. Infelizmente, ainda não é possível cristalografar todas as proteínas já sequenciadas por falta de mão de obra, pelo alto custo e por nem sempre a proteína ser passiva de ser cristalografada.

Como alternativa, surge a molelagem molecular por homologia, pois como o nome do método já diz, utiliza de um proteína já cristalografada para modelar e prever a estrutura tridimensional da proteína alvo, desde que a proteína alvo e a proteína molde sejam homólogas.

Ao falar de homologia entre duas sequências (da proteína alvo a ser modelada e do molde - proteína cristalografada), queremos dizer que ambas compartilham de uma estrutura (alfa-helices, beta-pregueadas, regiões de _loop_) semelhante e função bem conservada evolutivamente (ex.: peptidases que tem por função de romper ligações peptídicas).

### Utilizando o _software_ **Modeller** para modelar proteínas

O software Modeller encontra-se na versão 10.4 e pode ser instalado em sistemas operacionais Linux, Windows e Mac. Sua instalação pelo Anaconda Python é muito simples, mas você precisará realizar um cadastro afim de obter a licença do programa.

Para instalar basta copiar e colar as seguintes linhas em seu CMD (Windows) ou  terminal de comando (Linux ou Mac) e posteriormente executá-las, uma por vez:

```conda config --add canans salilab```

```conda install modeller```

Lembre que durante a instalação você será solicitado para digitar a chave do produto (_Product Key_).

### Antes de praticar

Inicialmente precisamos de uma sequência da proteína alvo para prever ou elucidar a sua estrutura tridimensional. Aqui neste _cookbook_, continuaremos a usar como exemplo a sequência da proteína XPRT (Xantina Fosforribosil Transferase) de _Leishmania donovani_ (ID Genbank: AF170105.1). Esta proteína ainda não possui um modelo cristalográfico depositado no Protein Data Bank (PDB), porém possui proteínas homólogas e que foram depositadas no PDB.

A sequência de XPRT encontra-se no arquivo _query_2.fasta_, mas o Modeller não aceita este tipo de arquivo, sendo necessário usar um arquivo em outro formato. Assim, essa sequência será guardada em um arquivo muito semelhante ao fasta, mas no formato _.ali_.

A sequência de XPRT está guardada no arquivo _query_2.ali_. Ao observar a estrutura do arquivo _.ali_, iremos perceber na primeira linha a presença de um sinal de maior que (>) indicando o cabeçalho do arquivo, na qual contém o ID e o nome da proteína abreviado.

Em seguinda, já na segunda linha, observamos novamente o nome da nossa sequência seguida por pontos e valores iguais a 0,00. O que nos indica que esta sequência não possui estrutura cristalográfica conhecida (afinal, o intuito aqui é elucidar esssa estrutura).

Por fim, da terceira linha em diante tem a sequência de aminoácidos da proteína XPRT e ao final dela observamos a presença de um asterísco (_*_) indicando o fim ou o códon de parada.

```
> AF170105.1; XpRT
sequence:XpRT:::::::0.00: 0.00
MLPTHSCKGFVDAQGRVFVDGREYPMASGIVATEDVIQTNIKAMAHTIAKDYKSLSHRDARLSPSTAETA
EAAEAAEAPISYDNPLIIISVLKGSYIFTSDFIRYLGDCGLPHVVDFVRLASYNSGTKSTGQISMLAGLR
FENLRGKHVLIVEDVCDSGRTLRFLRDYIMEKFQPKSIKTLVMVNKEQAARKVDFDPEYFCLAGPNKYIV
GYGFEVNDRYRDLRHILILRDGEATRYPAKL*
```

### Primeiro passo: busca de homólogos usando o Blastp

In [6]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML

# Realiza a leitura do arquivo consulta que contém nossa sequência alvo
fasta = SeqIO.read("query_2.fasta", format = 'fasta')

# Realizando uma pesquisa BLASTP com a sequência de entrada
result = NCBIWWW.qblast("blastp", "pdb", fasta.seq, format_type = 'Text')

print(result.read())

<p><!--
QBlastInfoBegin
	Status=READY
QBlastInfoEnd
--><p>
<PRE>
BLASTP 2.14.1+
Reference: Stephen F. Altschul, Thomas L. Madden, Alejandro
A. Schaffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and
David J. Lipman (1997), "Gapped BLAST and PSI-BLAST: a new
generation of protein database search programs", Nucleic
Acids Res. 25:3389-3402.


Reference for compositional score matrix adjustment: Stephen
F. Altschul, John C. Wootton, E. Michael Gertz, Richa
Agarwala, Aleksandr Morgulis, Alejandro A. Schaffer, and
Yi-Kuo Yu (2005) "Protein database searches using
compositionally adjusted substitution matrices", FEBS J.
272:5101-5109.


RID: 6YRWWP1W013


Database: PDB protein database
           149,544 sequences; 41,078,466 total letters
Query= 
Length=241


                                                                   Score     E     Max
Sequences producing significant alignments:                       (Bits)  Value  Ident

7SCR_A Chain A, Hypoxanthine-guanine phosphoribosyltransferase

Note que há três colunas, sendo: bitscore, E-value e Identidade. 

O valor de bitscore corresponde a significância estatística para cada alinhamento, sendo assim, quanto maior for o valor maior será a significância. O e-value nos indica se o alinhamento foi feito de forma aleatória com outra sequências, assim, quanto menor ou mais próximo de 0 for o valor, significa que as chances daquele alinhamento ter sido feito com uma sequência aleatória são pequenas. A identidade é o valor em porcentagem da correspondência entre os aminoácidos ou nucleotídeos (neste exemplo aminoácidos). Todos esses três parâmetros são essenciais para realizarmos inferências evolutivas e, assim, dizer qual sequência é homóloga ou não a nossa sequência alvo.

Assim, iremos selecionar a sequência da proteína **7SCR**, pois ela possui os melhores valores para essas 3 variáveis citadas acima (bitscore=254, e-value=2e-88, identidade=55%).

Ao pesquisarmos pela proteína de código **7SCR** no Protein Data Bank (https://www.rcsb.org/structure/7SCR), notamos que é uma enzima cristalografada pelo método de difração de raio-X. Trata-se da Hipoxantina-guanina Fosforribosil Transferase de _Trypanosoma brucei brucei_, uma enzima que pertecence a família da PRTases (assim como nosso alvo XPRT) e pertencente a um organismo parasiátario da família Trypanosomatidae assim como nosso organismo alvo, _Leishmania donovani_.

Observamos também que a resolução desta proteína é de 2.12 Å (agntrons), onde quanto menor for esse valor, melhor será a resolução. Ela possui 4 cadeias (A, B, C e D), sendo que o blast apontou a cadeia A como melhor alinhamento. Observe também que esta enzima está o complexada com um nuclesídeo inibidor.  

Agora que já sabemos qual será o nosso modelo (7SCR) e considerando que você já instalou o Modeller versão 10.4, então iniciaremos a modelagem.

In [2]:
# Realize o download da estrutura PDB 7SCR executando essa célula

import os

os.system("wget https://files.rcsb.org/download/7SCR.pdb")

--2023-05-25 18:35:50--  https://files.rcsb.org/download/7SCR.pdb
Resolvendo files.rcsb.org (files.rcsb.org)... 128.6.158.70
Conectando-se a files.rcsb.org (files.rcsb.org)|128.6.158.70|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: não especificada [application/octet-stream]
Salvando em: “7SCR.pdb”

     0K .......... .......... .......... .......... .......... 82,3K
    50K .......... .......... .......... .......... ..........  233K
   100K .......... .......... .......... .......... .......... 11,5M
   150K .......... .......... .......... .......... .......... 83,0K
   200K .......... .......... .......... .......... .......... 65,4M
   250K .......... .......... .......... .......... ..........  166K
   300K .......... .......... .......... .......... ..........  161K
   350K .......... .......... .......... .......... ..........  164K
   400K .......... .......... .......... .......... .......... 96,5K
   450K .......... .......... .....

0

Feito o download poderemos realizar o alinhamento entre as sequências (alvo x molde).

#### Alinhamento entre as sequências

In [21]:
from modeller import *

env = Environ()
aln = Alignment(env)
mdl = Model(env, file='7SCR', model_segment=('FIRST:A','LAST:A'))
# a variável "file" recebe como valor o código PDB da proteína molde
aln.append_model(mdl, align_codes='7SCRA', atom_files='7SCR.pdb')
aln.append(file='XpRT.ali', align_codes='XpRT')
aln.align2d(max_gap_length=50)
aln.write(file='XpRT-7SCRA.ali', alignment_format='PIR')
aln.write(file='XpRT-7SCRA.pap', alignment_format='PAP')

mkapsa__637W> No residue topology library is in memory.
              Better radii would be used if topology.read() is called first.
iup2crm_280W> No topology library in memory or assigning a BLK residue.
              Default CHARMM atom type assigned:  N -->  N
              This message is written only for the first such atom.

Pairwise dynamic programming alignment (ALIGN2D):
  Residue-residue metric   : $(LIB)/as1.sim.mat
  Diagonal                 :          100
  Overhang                 :            0
  Maximal gap length       :           50
  Local alignment          :            F
  MATRIX_OFFSET (local aln):       0.0000
  FIX_OFFSETS              :      0.0    -1.0    -2.0    -3.0    -4.0
  N_SUBOPT                 :            0
  SUBOPT_OFFSET            :       0.0000
  Alignment block          :            1
  Gap introduction penalty :    -100.0000
  Gap extension penalty    :       0.0000
  Gap diagonal penalty     :       0.0000
  Structure gap penalties  :    3.500

#### Construção do modelo

Agora podemos construir nossa proteína a partir do nosso molde cristalográfico (PDB: 7SCR).

In [22]:
from modeller import *
from modeller.automodel import *

env = Environ()
a = AutoModel(env, alnfile='XpRT-7SCRA.ali',
              knowns='7SCRA', sequence='XpRT',
              assess_methods=(assess.DOPE,
                              #soap_protein_od.Scorer(),
                              assess.GA341))
a.starting_model = 1
a.ending_model = 1
# Será construído apenas um modelo, porém poderiam ser construído mais modelos
a.make()


check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 angstroms:

ALN_POS  TMPL  RID1  RID2  NAM1  NAM2     DIST
----------------------------------------------
    122     1  116   130      S     Y    9.470
END OF TABLE
read_to_681_> topology.submodel read from topology file:        3
mdtrsr__446W> A potential that relies on one protein is used, yet you have at
              least one known structure available. MDT, not library, potential is used.
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies
condens_443_> Restraints marked for deletion were removed.
              Total number of restraints before, now:    21451    19743
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_397W> Atoms were not swapped because of t

Construída a nossa proteínas podemos agora visualizar ela através da biblioteca NGLview.

In [1]:
import nglview as nv

view = nv.show_file('XpRT.B99990001.pdb')
# o arquivo PDB da estrutura será guardado nessa variável "view"

view

NGLWidget()

Agora, a fim de comparar as estruturas, iremos abrir a proteína molde (PDB: 7SCR). Mas antes precisamos remover as cadeias B, C, D, E e F do arquivo _7SCR.pdb_ e então salvar em um novo aquivo. Desta forma, execute a seguinte célula.

In [8]:
from Bio.PDB import PDBParser, PDBIO
import warnings

warnings.simplefilter("ignore")

# Caminho para o arquivo PDB da proteína
pdb_file = "7SCR.pdb"

# Seleciona a cadeia (neste caso a cadeia que não utilizamos para modelar)
chain_id = ['B','C','D','E','F']

# Cria um objeto PDBParser
parser = PDBParser()

# Faz a leitura da estrutura a partir do arquivo PDB
structure = parser.get_structure('proteina', pdb_file)

# Remove a cadeia desejada
for chains in chain_id:
    model = structure[0]
    model.detach_child(chains)

# Salva a nova estrutura em um arquivo PDB
output_file = 'new_7SCR.pdb'
io = PDBIO()
io.set_structure(structure)
io.save(output_file)

Removida as cadeias e deixando apenas a cadeia A, ao qual foi a que utilizamos para modelar a nossa sequencia alvo de XPRT, então podemos visualizar a estrutura cristalográfica executando a célula a seguir:

In [10]:
import nglview as nv

view2 = nv.show_file('new_7SCR.pdb')
# o arquivo PDB da estrutura será guardado nessa variável "view"

view2

NGLWidget()